# These numbers are relative

- https://adventofcode.com/2023/day/3

For this challenge, all you need to do is store the positions of the numbers and the positions of the symbols separately. Then, to find the part numbers, calculate the positions that surround the numbers and check if any of those represent a symbol.

I used complex numbers to represent the coordinates here, using the real and imaginary parts as _x_ and _y_ coordinates. Because summing two complex numbers is essentially _two_ sums, one for the real and one for the imaginary parts of the numbers, complex numbers make it super easy to apply offsets to the coordinates: just add the complex number with the right _x_ and _y_ offsets encoded as the real and imaginary parts.

A small `_offsets()` function generates the offsets for the coordinates that surround a number, given its length. When parsing the map, we store the original number coordinate (the location of its first digit) plus its length, so we can later on use this information to calculate all the surrounding positions. Simply check if any of those surrounding positions is a symbol.


In [1]:
import re
import typing as t
from itertools import chain

_schematic_components = re.compile(r"(\d+)|([^.\d])")


def _offsets(length: int) -> t.Iterator[complex]:
    dxs = range(-1, length + 1)
    yield from chain(  # above, before and after, below
        (dx + -1j for dx in dxs), (-1 + 0j, dxs[-1] + 0j), (dx + 1j for dx in dxs)
    )


class EngineSchematic:
    def __init__(self, map: str) -> None:
        symbols: dict[complex, str] = {}
        numbers: list[tuple[complex, int, int]] = []
        for y, line in enumerate(map.splitlines()):
            for match in _schematic_components.finditer(line):
                pos = match.start() + y * 1j
                number, symbol = match.groups()
                if number is not None:
                    numbers.append((pos, len(number), int(number)))
                else:
                    symbols[pos] = symbol
        self.symbols = symbols
        self.numbers = tuple(numbers)

    @property
    def part_numbers(self) -> t.Iterator[int]:
        symbols = self.symbols
        for pos, length, number in self.numbers:
            if any(pos + dxy in symbols for dxy in _offsets(length)):
                yield number


test_map = """\
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""
test_schematic = EngineSchematic(test_map)
assert 114 not in test_schematic.part_numbers
assert 58 not in test_schematic.part_numbers
assert sum(test_schematic.part_numbers) == 4361

In [2]:
import aocd

engine_map = EngineSchematic(aocd.get_data(day=3, year=2023))
print("Part 1:", sum(engine_map.part_numbers))

Part 1: 507214


# Get into gear

This is essentially the same excersise as before, but now we only continue with numbers that are next to a `*` gear symbol, and we need to check _all_ the positions, not just 'any' surrounding position for a number. That's because a given number might be adjacent to multiple gears!

The trick is to keep a dictionary of gear positions mapping to the numbers that are adjacent. Then, after processing all the numbers, only produce gear ratios for gears with exactly two numbers next to them.


In [3]:
from collections import defaultdict
from operator import mul


class GearEngineSchematic(EngineSchematic):
    @property
    def gear_ratios(self) -> t.Iterator[int]:
        symbols = self.symbols
        gear_nums: dict[complex, list[int]] = defaultdict(list)
        for pos, length, number in self.numbers:
            for dxy in _offsets(length):
                spos = pos + dxy
                if symbols.get(spos) == "*":
                    gear_nums[spos].append(number)
        for nums in gear_nums.values():
            if len(nums) == 2:
                yield mul(*nums)


test_gear_schematic = GearEngineSchematic(test_map)
assert list(test_gear_schematic.gear_ratios) == [16345, 451490]

In [4]:
engine_map = gear_schematic = GearEngineSchematic(aocd.get_data(day=3, year=2023))
print("Part 2:", sum(engine_map.gear_ratios))

Part 2: 72553319
